In [4]:
import seekpath
import numpy as np

# Define the `kVector` class

In [5]:
class kVector:
    """For k-vector search, given the input structure, the nucleus and satellite
    diffraction peaks.

    :param bravfSym: Bravais lattice symbol
    :param cell: a `3x3` list of floats (cell[0] is the first lattice vector, …)
                     The cell vectors should be corresponding to the primitive
                     following the ITA convention.
    :param positions: a `Nx3` list of floats with the atomic coordinates in
                      fractional coordinates (i.e., w.r.t. the cell vectors)
    :param numbers: a length-`N` list with integers identifying uniquely the
                    atoms in the cell
    :param nucPeaks: a `mx4` list for holding the nucleus diffraction peaks
                     (nucPeaks[0] has 4 entries, giving the hkl indeces
                     corresponding to the conventional unit cell setting (ITA),
                     together with the d-spacing)
    :param superPeaks: a length-`n` list for the satellite peak positions in
                       `d`.
    :param option: control the scope for k vector search in the Brillouin zone,
                   `0` for high symmetry points only, `1` for high symmetry and
                   edges, `2` for the whole Brillouin zone.
    """
    transMatrix = {
        "cP": np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
        "tP": np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
        "hP": np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
        "oP": np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
        "mP": np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
        "cF": 1/2 * np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]),
        "oF": 1/2 * np.array([[0, 1, 1], [1, 0, 1], [1, 1, 0]]),
        "cI": 1/2 * np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]),
        "tI": 1/2 * np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]),
        "oI": 1/2 * np.array([[-1, 1, 1], [1, -1, 1], [1, 1, -1]]),
        "hR": 1/3 * np.array([[2, -1, -1], [1, 1, -2], [1, 1, 1]]),
        "oC": 1/2 * np.array([[1, 1, 0], [-1, 1, 0], [0, 0, 2]]),
        "oA": 1/2 * np.array([[0, 0, 2], [1, 1, 0], [-1, 1, 0]]),
        "mC": 1/2 * np.array([[1, -1, 0], [1, -1, 1], [1, 1, -1]])
    }

    
    def __init__(
        self,
        bravfSym: str,
        cell: list,
        positions: list,
        numbers: list,
        nucPeaks: list,
        superPeaks: list,
        option: int = 0
    ):
        self.bravfSym = bravfSym
        self.cell = cell
        self.positions = positions
        self.numbers = numbers
        self.nucPeaks = nucPeaks
        self.superPeaks = superPeaks
        self.option = option
        

    def kpathFinder(self) -> dict:
        """Provided the structure inputs, the routine will be collecting the
        inputs into a structure tuple which will be fed into the `get_path`
        routine in the `seekpath` module. The special k points and the k-path
        will be returned.
        
        :return: the dictionary containing the k-points and k-path
        """
        structure = (self.cell, self.positions, self.numbers)
        k_info_tmp = seekpath.get_path(structure, True)
        k_info = {
            "point_coords": k_info_tmp["point_coords"],
            "path": k_info_tmp["path"]
        }
    
        return k_info


    def hklConvToPrim(self) -> list:
        """Convert the hkl indeces in the conventional cell setting to the
        primitive cell setting.

        :return: same structure as the `nucPeaks` list, with the hkl indeces
        being converted to the primitive cell setting.
        """
        nuc_prim_hkl = list()
        for nuc_peak in self.nucPeaks:
            prim_hkl_tmp = np.matmul(
                np.array([nuc_peak[:3]]),
                kVector.transMatrix[self.bravfSym]
            )
            prim_hkl_tmp = list(prim_hkl_tmp[0]).append(nuc_peak[3])
            nuc_prim_hkl.append(prim_hkl_tmp)

        return


    def pointOnVector(s_point: list, e_point: list, distance: float) -> list:
        """Grab the coordinate of a point on a vector specified by the starting
        and ending points. The distance from the point on the vector to the
        starting point should be given as the parameter.

        :param s_point: a list for the coordinate of the starting point
        :param e_point: a list for the coordinate of the ending point
        :param distance: the distance away from the starting point
        :return: the coordinate of the point on the vector
        """
        vec_length = np.sqrt(sum((x - y)**2 for x, y in zip(s_point, e_point)))

        xp = s_point[0] + distance / vec_length * (e_point[0] - s_point[0])
        yp = s_point[1] + distance / vec_length * (e_point[1] - s_point[1])
        zp = s_point[2] + distance / vec_length * (e_point[2] - s_point[2])

        return [xp, yp, zp]


    def k_opt_finder(self) -> list:
        """This is the kernel of the class, defining the method for searching
        over the Brillouin zone for optimal k vector that best explains the
        observed satellite peaks, given the already refined nucleus structure.

        :return: the list of top candidates of the optimal k vector. If the top
                 one is below the threshold, i.e., the distance between the
                 nominal and observed positions of those satellite peaks smaller
                 than the uncertainty of peak positions (which is determined by
                 the instrument resolution), only one candiate will be returned.
                 Otherwise, top 10 candidates will be returned. 
        """
        # First, we search over the high symmetry points
        hs_points = self.kpathFinder()["point_coords"]
        


# Testing

In [6]:
cell = [
    [2.0626889900, 2.6083333300, -2.0959087400],
    [1.9072983400, 2.9395873000, 1.8656656300],
    [6.5783827900, -3.2025691800, 1.4025826200]
]
positions = [
    [0.2293290750, 0.9643429050, 0.7541443700],
    [0.7706709250, 0.0356570950, 0.2458556300],
    [0.1294345050, 0.4304467200, 0.0623625850],
    [0.8705654950, 0.5695532800, 0.9376374150],
    [0.4456382750, 0.6359233100, 0.4306348850],
    [0.5543617250, 0.3640766900, 0.5693651150]
]
numbers = [i for i in range(len(positions))]

nucPeaks = [
    [0, 1, 1, 2.2]
]
superPeaks = [
    2.25
]

In [7]:
k_search = kVector("cP",
    cell,
    positions,
    numbers,
    nucPeaks,
    superPeaks
)

In [9]:
k_search.kpathFinder()["point_coords"]

{'GAMMA': [0.0, 0.0, 0.0],
 'Z': [0.0, 0.0, 0.5],
 'Y': [0.0, 0.5, 0.0],
 'Y_2': [0.0, -0.5, 0.0],
 'X': [0.5, 0.0, 0.0],
 'V_2': [0.5, -0.5, 0.0],
 'U_2': [-0.5, 0.0, 0.5],
 'T_2': [0.0, -0.5, 0.5],
 'R_2': [-0.5, -0.5, 0.5]}